<a href="https://colab.research.google.com/github/WilliamSawran/ABC/blob/master/TextMiningHW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/'My Drive'/

In [3]:
import sys
import os
from os import listdir
from os.path import isfile, join
import pathlib

#data_dir = pathlib.Path("/content/drive/'My Drive'/").parent
train_data_dir = pathlib.Path("/content/drive/My Drive/20_newsgroup/20news-bydate-train")
test_data_dir = pathlib.Path("/content/drive/My Drive/20_newsgroup/20news-bydate-test")

train_dirnames = os.listdir(train_data_dir)
print("Training Data:")
print("Number of directories:", len(train_dirnames))
print("Directory names:", train_dirnames)
filenames = os.listdir(train_data_dir / "rec.autos")
print("\tNumber of files in rec.autos:", len(filenames))
print("\tSome example filenames:", filenames[:5])
filenames = os.listdir(train_data_dir / "comp.sys.mac.hardware")
print("\tNumber of files in comp.sys.mac.hardware:", len(filenames))
print("\tSome example filenames:", filenames[:5])

test_dirnames = os.listdir(test_data_dir)
print("\nTesting Data:")
print("Number of directories:", len(test_dirnames))
print("Directory names:", test_dirnames)
filenames = os.listdir(test_data_dir / "rec.autos")
print("\tNumber of files in rec.autos:", len(filenames))
print("\tSome example filenames:", filenames[:5])
filenames = os.listdir(test_data_dir / "comp.sys.mac.hardware")
print("\tNumber of files in comp.sys.mac.hardware:", len(filenames))
print("\tSome example filenames:", filenames[:5])

print("\ntrain_folders")
train_folders = [f for f in train_dirnames]
print(train_folders)
print("\ntest_folders")
test_folders = [f for f in test_dirnames]
print(test_folders)

Training Data:
Number of directories: 2
Directory names: ['comp.sys.mac.hardware', 'rec.autos']
	Number of files in rec.autos: 594
	Some example filenames: ['101595', '101598', '101577', '101593', '101562']
	Number of files in comp.sys.mac.hardware: 578
	Some example filenames: ['50428', '50464', '50458', '50483', '50435']

Training Data:
Number of directories: 2
Directory names: ['rec.autos', 'comp.sys.mac.hardware']
	Number of files in rec.autos: 396
	Some example filenames: ['103270', '103037', '103439', '103417', '103339']
	Number of files in comp.sys.mac.hardware: 385
	Some example filenames: ['51965', '51984', '51982', '51997', '51978']

train_folders
['comp.sys.mac.hardware', 'rec.autos']

test_folders
['rec.autos', 'comp.sys.mac.hardware']


In [46]:
#creating a 2D list to store list of all files in different folders

train_files = []
for train_folder_name in train_folders:
  train_folder_path = join(train_data_dir, train_folder_name)
  for f in listdir(train_folder_path):
    train_file_path = join(train_data_dir, train_folder_name, f)
    train_files.append(train_file_path)

test_files = []
for test_folder_name in test_folders:
  test_folder_path = join(test_data_dir, test_folder_name)
  for f in listdir(test_folder_path):
    test_file_path = join(test_data_dir, test_folder_name, f)
    test_files.append(test_file_path)



In [40]:
#checking total no. of files gathered
print(len(train_files))
print(train_files)

#checking total no. of files gathered
print(len(test_files))
print(test_files)

1172
['/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50428', '/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50464', '/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50458', '/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50483', '/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50435', '/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50500', '/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50462', '/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50501', '/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50422', '/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50479', '/content/drive/My Drive/20_newsgroup/20news-bydate-train/comp.sys.mac.hardware/50441', '/content/drive/My Drive/2

In [7]:
#making an array containing the classes each of the documents belong to
train_Y = []
for train_folder_name in train_folders:
  train_folder_path = join(train_data_dir, train_folder_name)
  num_of_files= len(listdir(train_folder_path))
  for i in range(num_of_files):
    train_Y.append(train_folder_name)

#making an array containing the classes each of the documents belong to
test_Y = []
for test_folder_name in test_folders:
  test_folder_path = join(test_data_dir, test_folder_name)
  num_of_files= len(listdir(test_folder_path))
  for i in range(num_of_files):
    test_Y.append(test_folder_name)

print(len(train_Y))
print(len(test_Y))

1172
781


# splitting the data into train test.

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
#a simple helper function to convert a 2D array to 1D, without using numpy
def flatten(list):
  new_list = []
  for i in list:
    for j in i:
      new_list.append(j)
  return new_list

In [10]:
!python -m spacy download en_core_web_lg



     |████████████████████████████████| 827.9 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [11]:
import spacy
import string
from bs4 import BeautifulSoup
import re
from spacy import displacy
from collections import Counter
#import en_core_web_lg
nlp = spacy.load('en_core_web_lg')


list_of_words = []

for document in train_files:
  #load document
  text = open(document, 'r', encoding="ISO-8859-1")
  text = BeautifulSoup(text, 'html.parser').get_text()
  text = re.sub(r'…', '...', text)
  text = re.sub(r'[`‘’‛⸂⸃⸌⸍⸜⸝]', "'", text)
  text = re.sub(r'[„“]|(\'\')|(,,)', '"', text)
  text = re.sub(r'\|>', '', text)
  #text = re.sub(r'\s+', ' ', text).strip()
  #text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
  document = nlp(text)
  #print(document)
  tokens = []
  for sent in document.sents:
      for token in sent:
        if token.is_alpha and len(token)>2 and not token.is_stop:
          #if token.IS_PUNCT and if not token.is_stop::
          tokens.append(token.lemma_.lower())
  list_of_words.append(flatten(tokens))


In [12]:
len(list_of_words)

1172

In [14]:
len(flatten(list_of_words))

620061

In [15]:
import numpy as np
np_list_of_words = np.asarray(flatten(list_of_words))
len(np_list_of_words)

620061

In [16]:
#finding the number of unique words that we have extracted from the documents
words, counts = np.unique(np_list_of_words, return_counts=True)
len(words)

32

In [17]:
#sorting the unique words according to their frequency
freq, wrds = (list(i) for i in zip(*(sorted(zip(counts, words), reverse=True))))

In [18]:
#deciding the no. of words to use as feature
n = 5000
features = wrds[0:n]

In [20]:
#creating a dictionary that contains each document's vocabulary and ocurence of each word of the vocabulary
dictionary = {}
doc_num = 1
for doc_words in list_of_words:
  #print(doc_words)
  np_doc_words = np.asarray(doc_words)
  w, c = np.unique(np_doc_words, return_counts=True)
  dictionary[doc_num] = {}
  for i in range(len(w)):
    dictionary[doc_num][w[i]] = c[i]
  doc_num = doc_num + 1

In [21]:
#now we make a 2D array having the frequency of each word of our feature set in each individual documents
X_train = []
for k in dictionary.keys():
  row = []
  for f in features:
    if(f in dictionary[k].keys()):
      #if word f is present in the dictionary of the document as a key, its value is copied
      #this gives us no. of occurences
      row.append(dictionary[k][f])
    else:
      #if not present, the no. of occurences is zero
      row.append(0)
  X_train.append(row)

In [ ]:
#we convert the X and Y into np array for concatenation and conversion into
dataframe
X_train = np.asarray(X_train)
train_Y = np.asarray(train_Y)

In [22]:
len(X_train)

1172

In [23]:
len(train_Y)

1172

# we'll make our test data by performing the same operations as we did for train data.

In [48]:
list_of_words_test = []

for document in test_files:
  #load document
  text = open(document, 'r', encoding="ISO-8859-1")
  text = BeautifulSoup(text, 'html.parser').get_text()
  text = re.sub(r'…', '...', text)
  text = re.sub(r'[`‘’‛⸂⸃⸌⸍⸜⸝]', "'", text)
  text = re.sub(r'[„“]|(\'\')|(,,)', '"', text)
  text = re.sub(r'\|>', '', text)
  #text = re.sub(r'\s+', ' ', text).strip()
  #text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
  document = nlp(text)
  #print(document)
  tokens = []
  for sent in document.sents:
      for token in sent:
        if token.is_alpha and len(token)>2 and not token.is_stop:
          #if token.IS_PUNCT and if not token.is_stop::
          tokens.append(token.lemma_.lower())
  list_of_words_test.append(flatten(tokens))

In [50]:
dictionary_test = {}
doc_num = 1
for doc_words in list_of_words_test:
  #print(doc_words)
  np_doc_words = np.asarray(doc_words)
  w, c = np.unique(np_doc_words, return_counts=True)
  dictionary_test[doc_num] = {}
  for i in range(len(w)):
    dictionary_test[doc_num][w[i]] = c[i]
  doc_num = doc_num + 1

In [51]:
#now we make a 2D array having the frequency of each word of our feature set in each individual documents
X_test = []
for k in dictionary_test.keys():
  row = []
  for f in features:
    if(f in dictionary_test[k].keys()):
      #if word f is present in the dictionary of the document as a key, its value is copied
      #this gives us no. of occurences
        row.append(dictionary_test[k][f])
    else:
      #if not present, the no. of occurences is zero
      row.append(0)
  X_test.append(row)

In [52]:
X_test = np.asarray(X_test)
test_Y = np.asarray(test_Y)

In [53]:
len(X_test)

781

In [54]:
len(test_Y)

781

# performing Text Classification using sklearn's Multinomial Naive Bayes.

In [55]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, train_Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [56]:
Y_predict = clf.predict(X_test)

In [59]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(test_Y, Y_predict))

                       precision    recall  f1-score   support

comp.sys.mac.hardware       0.77      0.66      0.71       385
            rec.autos       0.71      0.80      0.75       396

             accuracy                           0.73       781
            macro avg       0.74      0.73      0.73       781
         weighted avg       0.74      0.73      0.73       781

